# 第二次作业_02_2.8

> 新雅62/CDIE6 2016013327项雨桐

## 0. 导入包和数据

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
data_train = pd.read_table('HomeworkData/prostate_train.txt')
data_test = pd.read_table('HomeworkData/prostate_test.txt')

## 1. 检查数据

In [3]:
data_train.head()

,lcavol,lweight,lbph,svi,lpsa
0,-0.579818,2.769459,-1.386294,0,-0.430783
1,-0.994252,3.319626,-1.386294,0,-0.162519
2,-0.510826,2.691243,-1.386294,0,-0.162519
3,-1.203973,3.282789,-1.386294,0,-0.162519
4,0.751416,3.432373,-1.386294,0,0.371564


In [4]:
data_test.head()

,lcavol,lweight,lbph,svi,lpsa
0,0.737164,3.473518,0.615186,0,0.765468
1,-0.776529,3.539509,-1.386294,0,1.047319
2,0.223144,3.244544,-1.386294,0,1.047319
3,1.205971,3.442019,-1.386294,0,1.398717
4,2.059239,3.501043,1.474763,0,1.658228


In [5]:
data_train.isnull().sum()

lcavol     0
lweight    0
lbph       0
svi        0
lpsa       0
dtype: int64

In [6]:
data_test.isnull().sum()

lcavol     0
lweight    0
lbph       0
svi        0
lpsa       0
dtype: int64

数据没有缺损或异常，可以直接用于回归分析

## 2. 准备数据

### 2.1 无交叉

In [7]:
X_train = (np.array([data_train['lcavol'], data_train['lweight'], data_train['lbph'], data_train['svi']])).T
y_train = data_train['lpsa'].values
X_test = (np.array([data_test['lcavol'], data_test['lweight'], data_test['lbph'], data_test['svi']])).T
y_test = data_test['lpsa'].values

### 2.2 有交叉

In [8]:
x12 = (np.multiply(data_train['lcavol'].values,data_train['lweight'].values)).reshape(1,-1)
x13 = (np.multiply(data_train['lcavol'].values,data_train['lbph'].values)).reshape(1,-1)
x14 = (np.multiply(data_train['lcavol'].values,data_train['svi'].values)).reshape(1,-1)
x23 = (np.multiply(data_train['lweight'].values,data_train['lbph'].values)).reshape(1,-1)
x24 = (np.multiply(data_train['lweight'].values,data_train['svi'].values)).reshape(1,-1)
x34 = (np.multiply(data_train['lbph'].values,data_train['svi'].values)).reshape(1,-1)

X_crossed_train = np.array([data_train['lcavol'], data_train['lweight'], data_train['lbph'], data_train['svi']])
X_crossed_train = np.concatenate((X_crossed_train,x12,x13,x14,x23,x24,x34))
X_crossed_train = X_crossed_train.T

In [9]:
x12 = (np.multiply(data_test['lcavol'].values,data_test['lweight'].values)).reshape(1,-1)
x13 = (np.multiply(data_test['lcavol'].values,data_test['lbph'].values)).reshape(1,-1)
x14 = (np.multiply(data_test['lcavol'].values,data_test['svi'].values)).reshape(1,-1)
x23 = (np.multiply(data_test['lweight'].values,data_test['lbph'].values)).reshape(1,-1)
x24 = (np.multiply(data_test['lweight'].values,data_test['svi'].values)).reshape(1,-1)
x34 = (np.multiply(data_test['lbph'].values,data_test['svi'].values)).reshape(1,-1)

X_crossed_test = np.array([data_test['lcavol'], data_test['lweight'], data_test['lbph'], data_test['svi']])
X_crossed_test = np.concatenate((X_crossed_test,x12,x13,x14,x23,x24,x34))
X_crossed_test = X_crossed_test.T

## 3. 回归拟合

### 3.1 无交叉

In [10]:
model = LinearRegression() 
model.fit(X_train, y_train)

print('Regression Result:')
print(model.coef_, model.intercept_)

Regression Result:
[ 0.50552085  0.5388292   0.1400111   0.67184865] -0.325921234088


### 3.2 有交叉

In [11]:
model_crossed = LinearRegression() 
model_crossed.fit(X_crossed_train, y_train)

print('Regression Result:')
print(model_crossed.coef_, model_crossed.intercept_)

Regression Result:
[ 0.75872914  0.72396096  1.23365227  4.35858042 -0.0709226   0.04191441
 -0.1955892  -0.30074332 -0.87112746 -0.21748648] -0.882982674068


## 4. 预测结果与性能指标

In [12]:
y_pred = model.predict(X_test)
y_crossed_pred = model_crossed.predict(X_crossed_test)

print('预测值（无交叉）')
print(y_pred)
print('预测值（有交叉）')
print(y_crossed_pred)

预测值（无交叉）
[ 2.00449631  0.99462147  1.34104092  1.94428589  2.80801432  2.06887115
  2.13115131  1.80905482  2.05487204  1.24572611  2.85937501  2.19334314
  2.4753094   2.67626965  2.24731416  2.40617803  2.10025142  3.21743353
  2.96871831  1.51641713  3.58665261  3.7662857   2.48019576  2.66356745
  2.82721828  3.08095822  2.94509462  3.25201811  3.45156972  4.30428439]
预测值（有交叉）
[ 2.14470272  1.09584489  1.21343833  1.88427535  3.09655859  2.19905111
  2.26267865  1.82023638  2.12601095  1.13949148  2.81743808  2.24199304
  2.60428986  2.70596052  2.18763124  2.608396    2.24694355  3.24005007
  2.84124502  1.16026582  3.49567989  3.32595821  2.46003231  2.79059492
  3.38275638  3.66522235  2.96230812  3.41207126  3.75107586  3.85221037]


### 4.1 $R^2$

In [13]:
print('R^2（无交叉）')
print(model.score(X_test, y_test))
print('R^2（有交叉）')
print(model_crossed.score(X_crossed_test, y_test))

R^2（无交叉）
0.565250282204
R^2（有交叉）
0.493714539887


### 4.2 MSE

In [14]:
print('MSE（无交叉）')
print(mean_squared_error(y_test,y_pred))
print('MSE（有交叉）')
print(mean_squared_error(y_test,y_crossed_pred))

MSE（无交叉）
0.45633212204
MSE（有交叉）
0.531419133618
